# Logistica S.A. - Cálculo de Costos Cloud

In [26]:
import pandas as pd

## Listas de Precios segun AWS Price Calculator

In [27]:

# Configuración combinada de las instancias EC2, RDS, Backup y Lambda en un solo diccionario
precios = {
    'ec2_grande': {
        'tipo_instancia': 'm5.large',
        'precio_por_hora': 0.096,  # Precio aproximado por hora de una instancia grande
        'horas_por_mes': 24 * 30  # 24 horas durante 30 días
    },
    'ec2_mediana': {
        'tipo_instancia': 't3.medium',
        'precio_por_hora': 0.0464,  # Precio por hora de una instancia mediana
        'horas_por_mes': 24 * 30
    },
    'rds_postgres': {
        'tipo_instancia': 'db.t3.medium',
        'precio_por_hora': 0.145,  # Precio por hora de una instancia RDS
        'almacenamiento_gb': 100,  # Almacenamiento en GB
        'precio_almacenamiento_por_gb': 0.23,  # Precio por GB de almacenamiento RDS
        'horas_por_mes': 24 * 30
    },
    's3_almacenamiento': {
        'almacenamiento_s3_gb': 1000,  # Almacenamiento en S3 para backups
        'precio_almacenamiento_s3_por_gb': 0.023,  # Precio por GB en S3 Standard
        'tipo_almacenamiento': 'S3 Standard (1000 GB)'
    },
    'lambda': {
        'requests_per_day': 1000,
        'execution_time_seconds': 5,
        'memory_gb': 1,
        'storage_gb_per_day': 2,
        'costo_por_gb_segundo': 0.0000166667,  # Costo por GB-segundo de ejecución
        'costo_por_gb_dia': 0.0002  # Costo por GB por día de almacenamiento
    }
}

In [28]:
pd.DataFrame(precios)

,ec2_grande,ec2_mediana,rds_postgres,s3_almacenamiento,lambda
tipo_instancia,m5.large,t3.medium,db.t3.medium,NaN,NaN
precio_por_hora,0.096,0.0464,0.145,NaN,NaN
horas_por_mes,720,720,720,NaN,NaN
almacenamiento_gb,NaN,NaN,100,NaN,NaN
precio_almacenamiento_por_gb,NaN,NaN,0.23,NaN,NaN
almacenamiento_s3_gb,NaN,NaN,NaN,1000,NaN
precio_almacenamiento_s3_por_gb,NaN,NaN,NaN,0.023,NaN
tipo_almacenamiento,NaN,NaN,NaN,S3 Standard (1000 GB),NaN
requests_per_day,NaN,NaN,NaN,NaN,1000.000000
execution_time_seconds,NaN,NaN,NaN,NaN,5.000000


## Funciones de Cálculo de Precios

In [29]:

# Función para calcular los costos de EC2, RDS, Backup y Lambda
def calcular_costos_cloud(precios):
    # Calculo de costos EC2 y RDS
    costo_ec2_grande = precios['ec2_grande']['precio_por_hora'] * precios['ec2_grande']['horas_por_mes']
    costo_ec2_mediana = precios['ec2_mediana']['precio_por_hora'] * precios['ec2_mediana']['horas_por_mes']
    costo_rds_postgres = (precios['rds_postgres']['precio_por_hora'] * precios['rds_postgres']['horas_por_mes']) + \
                         (precios['rds_postgres']['almacenamiento_gb'] * precios['rds_postgres']['precio_almacenamiento_por_gb'])
   
    
    # Calculo del costo del backup
    costo_almacenamiento = precios['s3_almacenamiento']['almacenamiento_s3_gb'] * precios['s3_almacenamiento']['precio_almacenamiento_s3_por_gb']
    
    # Calculo del costo de Lambda
    # Lambda cost formula: execution cost (GB-seconds) + storage cost
    requests_per_day = precios['lambda']['requests_per_day']
    execution_time_seconds = precios['lambda']['execution_time_seconds']
    memory_gb = precios['lambda']['memory_gb']
    storage_gb_per_day = precios['lambda']['storage_gb_per_day']
    
    # Calcular el costo de ejecución en GB-segundos
    gb_seconds_per_request = memory_gb * execution_time_seconds
    gb_seconds_per_day = requests_per_day * gb_seconds_per_request
    gb_seconds_per_month = gb_seconds_per_day * 30  # Suponiendo 30 días por mes
    costo_lambda_execution = gb_seconds_per_month * precios['lambda']['costo_por_gb_segundo']
    
    # Calcular el costo de almacenamiento
    costo_lambda_storage = storage_gb_per_day * 30 * precios['lambda']['costo_por_gb_dia']
    
    costo_lambda_total = costo_lambda_execution + costo_lambda_storage

    # Costo total
    costo_total = costo_ec2_grande + costo_ec2_mediana + costo_rds_postgres  + costo_almacenamiento + costo_lambda_total

    return {
        'costo_ec2_grande': costo_ec2_grande,
        'costo_ec2_mediana': costo_ec2_mediana,
        'costo_rds_postgres': costo_rds_postgres,
        'costo_s3_almacenamiento': costo_almacenamiento,
        'costo_lambda': costo_lambda_total,
        'costo_total': costo_total
    }

# Función para generar la lista técnica en un formato estructurado
def generar_lista_tecnica(precios):
    lista_servicios = {
        "Instancia EC2 Grande": {
            "Tipo Instancia": precios['ec2_grande']['tipo_instancia'],
            "Precio por Hora": f"{precios['ec2_grande']['precio_por_hora']} USD"
        },
        "Instancia EC2 Mediana": {
            "Tipo Instancia": precios['ec2_mediana']['tipo_instancia'],
            "Precio por Hora": f"{precios['ec2_mediana']['precio_por_hora']} USD"
        },
        "RDS Postgres": {
            "Instancias": precios['rds_postgres']['tipo_instancia'],
            "Almacenamiento": f"{precios['rds_postgres']['almacenamiento_gb']} GB",
            "Precio por Hora": f"{precios['rds_postgres']['precio_por_hora']} USD"
        },
        "S3 Almacenamiento de Modelos": {
            "Almacenamiento": precios['s3_almacenamiento']['tipo_almacenamiento'],
            "Almacenamiento en GB": f"{precios['s3_almacenamiento']['almacenamiento_s3_gb']} GB"
        },
        "AWS Lambda": {
            "Memoria": f"{precios['lambda']['memory_gb']} GB",
            "Requests Diarios": f"{precios['lambda']['requests_per_day']}",
            "Tiempo por Request": f"{precios['lambda']['execution_time_seconds']} segundos",
            "Almacenamiento Diario": f"{precios['lambda']['storage_gb_per_day']} GB"
        }
    }
    
    df_tecnica = pd.DataFrame.from_dict(lista_servicios, orient='index')
    return df_tecnica

# Función para generar la tabla financiera
def generar_tabla_financiera(costos):
    # Crear un DataFrame con los costos desglosados
    data = {
        'Recurso': ['EC2 Grande', 'EC2 Mediana', 'RDS Postgres', 'Almacenamiento S3', 'Lambda'],
        'Costo Mensual (USD)': [
            costos['costo_ec2_grande'], 
            costos['costo_ec2_mediana'], 
            costos['costo_rds_postgres'], 
            costos['costo_s3_almacenamiento'],
            costos['costo_lambda']
        ]
    }
    
    df_costos = pd.DataFrame(data)
    
    # Calcular el costo total y agregarlo a la tabla
    df_costos.loc[len(df_costos)] = ['Total', costos['costo_total']]
    
    return df_costos

In [30]:
# Llamada a la función para calcular costos
costos = calcular_costos_cloud(precios)

# Generar y mostrar la lista técnica en DataFrame
df_lista_tecnica = generar_lista_tecnica(precios)
display(df_lista_tecnica)

# Generar y mostrar la tabla financiera
tabla_financiera = generar_tabla_financiera(costos)
display(tabla_financiera)

,Tipo Instancia,Precio por Hora,Instancias,Almacenamiento,Almacenamiento en GB,Memoria,Requests Diarios,Tiempo por Request,Almacenamiento Diario
Instancia EC2 Grande,m5.large,0.096 USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Instancia EC2 Mediana,t3.medium,0.0464 USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RDS Postgres,NaN,0.145 USD,db.t3.medium,100 GB,NaN,NaN,NaN,NaN,NaN
S3 Almacenamiento de Modelos,NaN,NaN,NaN,S3 Standard (1000 GB),1000 GB,NaN,NaN,NaN,NaN
AWS Lambda,NaN,NaN,NaN,NaN,NaN,1 GB,1000,5 segundos,2 GB


,Recurso,Costo Mensual (USD)
0,EC2 Grande,69.120000
1,EC2 Mediana,33.408000
2,RDS Postgres,127.400000
3,Almacenamiento S3,23.000000
4,Lambda,2.512005
5,Total,255.440005
